In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# installing nest_asyncio to use twint in IPython environment (notebooks)
!pip install nest_asyncio 

# you might've to restart the session, in that case,
# don't execute this block again!

In [ ]:
%%shell
pip install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [ ]:
cp -a /root/.local/lib/python3.7/site-packages/. /usr/local/lib/python3.7/dist-packages/

In [ ]:
import twint as tw    # for scraping tweets
import nest_asyncio   # to use `twint` in notebooks
import nltk           # for pre-processing tweets
import re             # for pre-processing tweets

nltk.download('punkt') # For tokenizers
from nltk.tokenize import TweetTokenizer

tknz = TweetTokenizer()
nest_asyncio.apply()  # without this, `twint` won't work on Notebook

In [ ]:
# helper functions

# extract column names from twint data dump
def column_names():
    return tw.output.panda.Tweets_df.columns

# convert `twint` data dump to dataframe
def twint_to_pd(columns):
    return tw.output.panda.Tweets_df[columns]

# pre-processing tweets
def cleanText(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"www\S+", "", text)
    text = re.sub("-", " ", text)
    text = re.sub("\s+", " ", text)
    text = re.sub("\u2018", "X", text) 
    text = re.sub("\u2019", "X", text) 
    text = re.sub("\'", "X", text) 
    wordTokens_ = tknz.tokenize(text)
    wordTokens = list()
    for x in wordTokens_:
        x = ''.join([v for v in x if v.isalnum() or v == ' '])
        if len(x) > 0 and x != 'X':
            x = x.replace('X', '\'')
            wordTokens.append(x)
    return ' '.join(wordTokens)

In [ ]:
c = tw.Config()      # twint.Config object
c.Search = "@nytimes"
c.Lang= "en"         
c.Pandas = True 
c.Since = '2022-01-01'
c.Until = '2022-02-03'
tw.run.Search(c) 

In [ ]:
scrape_tweets(search_pattern="@nytimes", start_date='2022-03-03', end_date='2022-03-04')

In [ ]:
# for now, only collect the tweets
df = twint_to_pd(tw.output.panda.Tweets_df.columns)

In [ ]:
df

In [ ]:
# pre-processing the tweets

df['pptweet'] = df['tweet'].apply(cleanText)

In [ ]:
df

In [ ]:
df.to_csv('/content/gdrive/MyDrive/DL/Twitter/npr.csv', index=False)